# realfast analysis

In [ ]:
import logging
logger = logging.getLogger()
for hdlr in logger.handlers:
    logger.removeHandler(hdlr)
logging.basicConfig()

from bokeh.plotting import output_notebook
from rtpipe import interactive, nbpipeline
from ipywidgets import Button, Output, VBox
from IPython.display import display

# following needed for classifier
import os
import activegit
import numpy as np

In [ ]:
output_notebook()

In [ ]:
candsfile, noisefile, fileroot = interactive.initializenb()
statedir = fileroot+'.ipynb-state'

### Prepare data and noise plot

In [ ]:
loc, prop, d = interactive.read_candidates(candsfile, returnstate=True)
data = interactive.readdata(d=d, cands=(loc, prop))

### Calculate classifier score from latest training set

In [ ]:
try:
    agdir = os.environ['agdir']
except KeyError:
    home = os.environ['HOME']
    agdir = os.path.join(home, 'code', 'alnotebook')

if agdir:
    data['score'] = interactive.addclassifications(agdir, prop)
else:
    print('No agdir defined, so no score calculated.')

### Remove bad times and cands, then calculate indices

In [ ]:
plinds = {'cir': [], 'cro': [], 'edg': []}  # initialize plot symbols
nbpipeline.setFloat('threshold', default=15, min=5, max=20, description='Threshold to find bad times',
                   statedir=statedir)
nbpipeline.setText('ignorestr', default='', description='Times to ignore (comma-delimited)', statedir=statedir)

In [ ]:
interactive.filterdata(data=data, plinds=plinds, d=d, threshold=nbpipeline.read('threshold'),
                       ignorestr=nbpipeline.read('ignorestr'), thresh0=6., thresh1=7.)

### Generate candidate and noise plots

In [ ]:
nbpipeline.setDropdown('scaling', default='snrs_3', options=['snrs_3', 'snrs_5', 'score_5', 'score_7'],
                        description='Symbol size scaling ("source_powerlaw")',
                        statedir=statedir)
nbpipeline.setDropdown('plottype', default='all', options=['all', 'dmt', 'norm', 'loc', 'stat'],
                       description='Type of plot to make',
                       statedir=statedir)

In [ ]:
interactive.displayplot(data, plinds, nbpipeline.read('plottype'), nbpipeline.read('scaling'), fileroot, 
                        url_path='http://www.aoc.nrao.edu/~claw/plots')

In [ ]:
noiseplot = interactive.plotnoise(noisefile, candsfile, plot_width=900, plot_height=450)

### Comments

In [ ]:
nbpipeline.setText('comments', statedir=statedir)

In [ ]:
nbpipeline.list(statedir=statedir)

### Optional refinement analysis
#### Requires download data from archive into working directory